In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame,read_csv
from sklearn.utils import shuffle

In [21]:
df = read_csv('heart_failure_clinical_records_dataset.csv')
df.head(15)


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
5,90.0,1,47,0,40,1,204000.00,2.1,132,1,1,8,1
6,75.0,1,246,0,15,0,127000.00,1.2,137,1,0,10,1
7,60.0,1,315,1,60,0,454000.00,1.1,131,1,1,10,1
8,65.0,0,157,0,65,0,263358.03,1.5,138,0,0,10,1
9,80.0,1,123,0,35,1,388000.00,9.4,133,1,1,10,1


In [3]:
df['anaemia'] = np.where(df['anaemia'] == 1 ,True,False)
df['diabetes'] = np.where(df['diabetes'] == 1, True, False)
df['high_blood_pressure'] = np.where(df['high_blood_pressure'] == 1, True, False)
df['smoking'] = np.where(df['smoking'] == 1, True, False)
df['sex'] = np.where(df['sex'] == 1, 'Male','Female')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    bool   
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    bool   
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    bool   
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    object 
 10  smoking                   299 non-null    bool   
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: bool(4), float64(3), int64(5), object(1)
memory usage: 22.3+ K

In [5]:
X = df.drop('DEATH_EVENT', axis = 1)
y = df['DEATH_EVENT']
print(X.shape)
X.head(2)

(299, 12)


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,75.0,False,582,False,20,True,265000.00,1.9,130,Male,False,4
1,55.0,False,7861,False,38,False,263358.03,1.1,136,Male,False,6


In [6]:
import imblearn
from imblearn.over_sampling import SMOTENC
smote = SMOTENC(random_state=1,categorical_features=[0,1,3,5,9,10])
# oversample = SMOTE()
# X, y = oversample.fit_resample(X, y)
x_bal, y_bal = smote.fit_resample(X,y)
x_bal = DataFrame(x_bal, columns = X.columns)
print(Series(y_bal).value_counts())

1    203
0    203
Name: DEATH_EVENT, dtype: int64


In [7]:
x = pd.get_dummies(x_bal,columns = ['sex'], drop_first = True)
print(x.shape)
x.head()

(406, 12)


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,smoking,time,sex_Male
0,75.0,False,582,False,20,True,265000.00,1.9,130,False,4,1
1,55.0,False,7861,False,38,False,263358.03,1.1,136,False,6,1
2,65.0,False,146,False,20,False,162000.00,1.3,129,True,7,1
3,50.0,True,111,False,20,False,210000.00,1.9,137,False,7,1
4,65.0,True,160,True,20,False,327000.00,2.7,116,False,8,0


In [8]:
col = ['age','creatinine_phosphokinase','ejection_fraction',
       'platelets','serum_creatinine','serum_sodium','time',
       'anaemia','diabetes','high_blood_pressure','smoking',
       'sex_Male']

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

col_trans = ColumnTransformer(remainder='passthrough',
                              transformers = [('scaler',MinMaxScaler(),[0,2,4,6,7,8,10])])
trans = col_trans.fit_transform(x)
trans = DataFrame(trans,columns = col)
trans.head()

,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,time,anaemia,diabetes,high_blood_pressure,smoking,sex_Male
0,0.636364,0.071319,0.090909,0.290823,0.157303,0.485714,0.0,False,False,True,False,1
1,0.272727,1.0,0.363636,0.288833,0.067416,0.657143,0.007117,False,False,False,False,1
2,0.454545,0.015693,0.090909,0.16596,0.089888,0.457143,0.010676,False,False,False,True,1
3,0.181818,0.011227,0.090909,0.224148,0.157303,0.685714,0.010676,True,False,False,False,1
4,0.454545,0.017479,0.090909,0.365984,0.247191,0.085714,0.014235,True,True,False,False,0


In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(trans,y_bal, test_size = 0.2, random_state = 0)

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
grid = GridSearchCV(RandomForestClassifier(),
                {'max_depth':range(1,15), 'n_estimators': [50, 100,300],
                'max_features': ['auto', 'sqrt', 'log'], 
                'min_samples_split': [2,3,5,7,9],'min_samples_leaf': [1,2,4,6,8]},
                 cv = 3, n_jobs = 2, scoring = 'recall', verbose = 1)

grid.fit(x_train,y_train)

Fitting 3 folds for each of 3150 candidates, totalling 9450 fits


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
3150 fits failed out of a total of 9450.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3150 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py", line 450, in fit
    trees = Parallel(
  File "/usr/local/lib/python3.8/dist-packages/joblib/parallel.py", line 1085, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.8/dist-packages/joblib/parallel.

GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=2,
             param_grid={'max_depth': range(1, 15),
                         'max_features': ['auto', 'sqrt', 'log'],
                         'min_samples_leaf': [1, 2, 4, 6, 8],
                         'min_samples_split': [2, 3, 5, 7, 9],
                         'n_estimators': [50, 100, 300]},
             scoring='recall', verbose=1)

In [12]:
grid.best_params_

{'max_depth': 7,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 50}

In [13]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_features= 'auto',random_state = 3,
                                    min_samples_leaf = 1,
                                    min_samples_split = 2,n_estimators = 100)
classifier.fit(x_train, y_train)

RandomForestClassifier(random_state=3)

In [14]:
y_pred = classifier.predict(x_test)

In [15]:
from sklearn.metrics import classification_report, recall_score, accuracy_score,precision_score, f1_score, confusion_matrix
recall = recall_score(y_test,y_pred)
accuracy = accuracy_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
f1score = f1_score(y_test,y_pred)
print('the Recall for tuned RandomForest is:{}'.format(round(recall,4)))
print(f'the Accuracy for tuned RandomForest is:{round(accuracy,4)}')
print('the Precision for tuned RandomForest is: %s' %(round(precision,4)))
print(f'the F1_score for tuned RandomForest is:{round(f1score,4)}')

the Recall for tuned RandomForest is:0.9574
the Accuracy for tuned RandomForest is:0.939
the Precision for tuned RandomForest is: 0.9375
the F1_score for tuned RandomForest is:0.9474


In [16]:
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.94      0.91      0.93        35
           1       0.94      0.96      0.95        47

    accuracy                           0.94        82
   macro avg       0.94      0.94      0.94        82
weighted avg       0.94      0.94      0.94        82



In [17]:
matrix = confusion_matrix(y_test,y_pred)
print(matrix)

[[32  3]
 [ 2 45]]


In [18]:
import pickle
filename = 'myheartDisease_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [19]:
loaded_model = pickle.load(open('myheartDisease_model.sav', 'rb'))

In [20]:
# input_data = (50.0,1,168,0,38,1,276000.00,1.1,137,1,0,11)
input_data=(49.0,1,80,0,30,1,427000.00,1.0,138,0,0,12)
# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = loaded_model.predict(input_data_reshaped)


if (prediction[0] == 0):
  print(prediction)
  print('The person is not in a risk of heart disease')
else:
  print(prediction)
  print('The person is in risk of heart disease')

[0]
The person is not in a risk of heart disease


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
